In [1]:
import sys
sys.path.append('..')
from harang import vision, utils
import pandas as pd
from tqdm import tqdm
import cv2
import numpy as np
import os

c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' 

<IPython.core.display.Javascript object>

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/sample_submission.csv')
folds = utils.from_pickle('data/folds.pkl')

In [3]:
RESIZE = (320, 480)

In [4]:
def preprocess(path, is_train=True, orig_size=(1400, 2100)):

    if is_train:
        path = f'data/train_images/{orig_size[0]}_{orig_size[1]}/{path}'
    else:
        path = f'data/test_images/{orig_size[0]}_{orig_size[1]}/{path}'
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image = vision.np_resize(image, RESIZE)
    x = np.array(image, dtype=np.float32)
    return x

In [2]:
unlabeled = np.array([np.array(cv2.cvtColor(cv2.imread(f'data/unlabeled/{path}'), cv2.COLOR_BGR2RGB), dtype=np.uint8) for path in tqdm(os.listdir('data/unlabeled'))])
utils.to_pickle(f'data/unlabeled.pkl', unlabeled)

100%|███████████████████████████████████████████████████████████████████████████| 28726/28726 [01:17<00:00, 372.96it/s]


In [6]:
test = np.array([preprocess(path, is_train=False) for path in tqdm(test['Image_Label'].map(lambda x: x.split('_')[0]).values[::4])])
utils.to_pickle(f'data/test_{RESIZE}.pkl', test)

100%|██████████████████████████████████████████████████████████████████████████████| 3698/3698 [02:46<00:00, 22.18it/s]


In [5]:
x_data = np.array([preprocess(path, is_train=True) for path in tqdm(train['Image_Label'].map(lambda x: x.split('_')[0]).values[::4])])
y_data = np.array([vision.build_masks(rle, input_shape=(1400, 2100), reshape=RESIZE) for rle in tqdm(pd.read_pickle('data/train.pkl').values)])

100%|██████████████████████████████████████████████████████████| 5546/5546 [03:36<00:00, 25.59it/s]


In [6]:
utils.to_pickle(f'data/x_data.pkl', x_data)
utils.to_pickle(f'data/y_data.pkl', y_data)